<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

# Data Cleaning

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, date
from sklearn.cluster import KMeans

## Order Dataset

In [ ]:
order_df = pd.read_csv('olist_orders_dataset.csv')
order_df.info()

In [ ]:
order_df[order_df.duplicated()]

In [ ]:
order_df.drop(['order_status', 'order_delivered_carrier_date', 'order_approved_at'], axis='columns', inplace=True)
for column in ['order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']:
    order_df[column] = pd.to_datetime(order_df[column])
order_df.info()

In [ ]:
order_df.head()

## Payment Dataset

In [ ]:
payment_df = pd.read_csv('olist_order_payments_dataset.csv')
payment_df.info()

In [ ]:
payment_df[payment_df.duplicated()]

In [ ]:
payment_df.head()

In [ ]:
payment_df.describe()

In [ ]:
payment_df.nunique()

## Customer Dataset

In [ ]:
customer_df = pd.read_csv('olist_customers_dataset.csv')
customer_df.info()

In [ ]:
customer_df[customer_df.duplicated()]

In [ ]:
customer_df.drop(['customer_zip_code_prefix', 'customer_city'], axis='columns', inplace=True)
customer_df.columns

In [ ]:
customer_df.nunique()

In [ ]:
customer_df.head()

## Item Dataset

In [ ]:
item_df = pd.read_csv('olist_order_items_dataset.csv')

In [ ]:
item_df.info()

In [ ]:
item_df[item_df.duplicated()]

In [ ]:
item_df.drop(['seller_id',
              'shipping_limit_date',
              'price',
              'freight_value'], axis='columns', inplace=True)
item_df.columns

In [ ]:
item_df.head()

## Product Dataset

In [ ]:
product_df = pd.read_csv('olist_products_dataset.csv')

In [ ]:
product_df.info()

In [ ]:
product_df[product_df.duplicated()]

In [ ]:
product_df.drop(['product_name_lenght',
                 'product_description_lenght', 
                 'product_photos_qty', 
                 'product_weight_g',
                 'product_length_cm',
                 'product_height_cm',
                 'product_width_cm'], axis='columns', inplace=True)
product_df.columns

In [ ]:
product_df.fillna('other', inplace=True)
product_df.info()

In [ ]:
product_df.head()

In [ ]:
product_df.product_category_name.nunique()

## Translation Dataset

In [ ]:
translation_df = pd.read_csv('product_category_name_translation.csv')
translation_df.info()
translation_df.head()

## Review Dataset

In [ ]:
review_df = pd.read_csv('olist_order_reviews_dataset.csv')
review_df.info()

In [ ]:
review_df[review_df.duplicated()]

In [ ]:
review_df.drop(['review_creation_date', 'review_answer_timestamp'], axis='columns', inplace=True)
review_df.columns

In [ ]:
for columns in ['review_comment_title', 'review_comment_message']:
    # replace present values with 1
    review_df.loc[review_df[columns].notnull(), columns] = 1
    # replace null values with 0
    review_df.loc[review_df[columns].isna(), columns] = 0
review_df.info()

In [ ]:
review_df.head()

In [ ]:
review_df.nunique()

# Exploratory Data Analytics with Raw Data

In [ ]:
plt.bar(payment_df.groupby('payment_type').sum().index, 
        payment_df.groupby('payment_type').sum()['payment_value']/payment_df.groupby('payment_type').count()['payment_value']);

In [ ]:
plt.scatter(payment_df.payment_installments, payment_df.payment_value);

In [ ]:
plt.hist(review_df.review_score);

In [ ]:
df = pd.merge(payment_df, order_df, how='left', on='order_id')
df.head()

In [ ]:
df = pd.merge(df, customer_df, how='left', on='customer_id')
df.head()

In [ ]:
state_pay_series = df.groupby('customer_state').sum()['payment_value']/df.groupby('customer_state').count()['payment_value']
state_pay_series.sort_values(inplace=True, ascending=False)
plt.bar(state_pay_series.index, state_pay_series);

In [ ]:
df = pd.merge(df, item_df, how='right', on='order_id')
df.info()

In [ ]:
df = pd.merge(df, product_df, how='left', on='product_id')
df.head()

In [ ]:
#df = pd.merge(df, translation_df, how='left', on='product_category_name')
#df.drop('product_category_name', axis='columns', inplace=True)
#df.head()

In [ ]:
payment_by_category = df.groupby('product_category_name')['payment_value'].sum()
payment_by_category.sort_values(ascending=False, inplace=True)
plt.figure(figsize=(12,20))
plt.barh(payment_by_category.index, payment_by_category)

In [ ]:
df = pd.merge(df, review_df, how='right', on='order_id')
df.head()

# Feature Engineering

In [ ]:
df['purchase_date'] = df['order_purchase_timestamp'].dt.date
df['delay'] = (df.order_delivered_customer_date - df.order_estimated_delivery_date).dt.days
df.drop(['order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date'], axis='columns', inplace=True)
df.head()

In [ ]:
plt.hist(df.delay, bins=50);

In [ ]:
plt.scatter(df.purchase_date, df.delay);

In [ ]:
df1 = df[(date(2016,10,1) <= df.purchase_date) & (df.purchase_date < date(2017,10,1))]
df2 = df[(date(2017,10,1) <= df.purchase_date) & (df.purchase_date < date(2018,10,1))]

# Exploratory Data Analytics

In [ ]:
df1.head()

In [ ]:
feature_df = df1.groupby('customer_unique_id').agg({'purchase_date': 'max', 
                                                    'order_id': 'count',
                                                    'payment_value': 'sum',
                                                    'delay': 'mean',
                                                    'payment_installments': 'mean',
                                                    'review_score': 'mean',
                                                    'review_comment_title': 'mean',
                                                    'review_comment_message': 'mean'})
feature_df.columns = ['recency', 'frequency', 'monetary', 'delay', 'installments', 'review_score', 'comment_title', 'comment_message']
feature_df

In [ ]:
type_series = df1.groupby('customer_unique_id')['payment_type'].agg(lambda x:x.value_counts().index[0])
type_series

In [ ]:
df1 = pd.merge(df1, type_series, left_on='customer_unique_id', right_index=True)
df1

In [ ]:
df1.info()

In [56]:
category_series = df1.groupby('customer_unique_id')['product_category_name'].agg(lambda x:x.value_counts().index[0])
category_series

customer_unique_id
0000f46a3911fa3c0805444483337064                      papelaria
0005e1862207bf6ccc02e4228effd9a0    fashion_bolsas_e_acessorios
0006fdc98a402fceb4eb0ee528f6a8d4                cama_mesa_banho
000a5ad9c4601d2bbdd9ed765d5213b3                   beleza_saude
000bfa1d2f1a41876493be685390d6d3    fashion_bolsas_e_acessorios
                                               ...             
fff3a9369e4b7102fab406a334a678c3               malas_acessorios
fff699c184bcc967d62fa2c6171765f7                 consoles_games
fffcf5a5ff07b0908bd4e2dbc735a684                   beleza_saude
ffff371b4d645b6ecea244b27531430a                     automotivo
ffffd2657e2aad2907e67c3e9daecbeb                     perfumaria
Name: product_category_name, Length: 26455, dtype: object

In [57]:
df1 = pd.merge(df1, category_series, left_on='customer_unique_id', right_index=True)
df1

,order_id,payment_sequential,payment_type_x,payment_installments,payment_value,customer_id,customer_unique_id,customer_state,order_item_id,product_id,product_category_name_x,review_id,review_score,review_comment_title,review_comment_message,purchase_date,delay,payment_type_y,product_category_name_y
8,771ee386b001f06208a7419e4fc1bbd7,1.0,credit_card,1.0,81.16,206c0fcc717333d169ff95933fb47341,9a3526eee723accd014ec683afe5bf20,MG,1.0,42a2c92a0979a949ca4ea89ec5c7b934,utilidades_domesticas,35f54313f077af011b5e3a8f0dafb9d7,5,0,1,2017-06-23,-14.0,credit_card,utilidades_domesticas
9,3d7239c394a212faae122962df514ac7,1.0,credit_card,3.0,51.84,4d7a9b8bba459dce09d1b8fa637a0cba,9db67268a3fee1d4b13faaeb90af07c0,SP,1.0,92e16cf2fab490ab9ae87a305d5f8f7f,perfumaria,1e23005084e0ca486cb6be7f343ab1ef,4,0,0,2017-06-05,-3.0,credit_card,perfumaria
11,0573b5e23cbd798006520e1d5b4c6714,1.0,boleto,1.0,51.95,c4e17eff78176dfe3401f03db0346f85,511c345b0383af6ef25406c564e4d6b6,SP,1.0,5299e0a0336b5839e2a0ccb0d3d589d1,artes,327425bbda502d806f410c544b730a23,4,0,1,2017-07-04,-13.0,boleto,artes
17,cf95215a722f3ebf29e6bbab87a29e61,1.0,credit_card,5.0,102.66,e2a8d07201639004b090670ce5efe355,ec103b892f579de25189dda196a4c5b3,MG,1.0,57d2fa6882b789ad55e86074398826c4,perfumaria,a49bf122101cc876bb5cdf5d1dfca33c,5,0,1,2017-08-25,-11.0,credit_card,perfumaria
18,769214176682788a92801d8907fa1b40,1.0,credit_card,4.0,105.28,2b01b5fb8141bbb9fcdc853a0e26da83,9b61cfcf9bc923fed1c1c8d38fab5a1c,ES,1.0,99a4788cb24856965c36a24e339b6058,cama_mesa_banho,b438431e04ae1ad0cb7ef798fb609742,5,0,0,2017-06-29,-5.0,credit_card,cama_mesa_banho
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118293,807f4ef550f104708e2190eca7e4c37a,1.0,credit_card,3.0,205.71,993f60535be5935dba6ea0fd2e71c092,3146229460ce28f41121b47929b046b0,MA,1.0,7afa8c8eca2a26b599823eed4975a820,utilidades_domesticas,fcfdc4552257b85fd2a7c20182a61a7c,5,0,1,2017-02-17,-22.0,credit_card,utilidades_domesticas
118294,4b186efcda6f283e9c4a71332f6d3ad0,1.0,credit_card,1.0,57.51,cb58cd7d199d31c8320261119c8eca20,8247de12f35a00cf569c5b6a00a746ec,PR,1.0,7926af7d3e25dce27c3eee0279afb530,pet_shop,e82392dcd96a4ac41981eddcc6df5611,3,0,0,2017-04-07,-15.0,credit_card,pet_shop
118295,c11717d999ffc510a6441df946f0a94d,1.0,credit_card,9.0,96.90,f04581c79df64e4152c87007e29da2f4,b388a93f3fdda4dc124231dfb9bd06d3,PR,1.0,cd2c03056720c7de4d6a59c1167bdee0,fashion_bolsas_e_acessorios,80efb32ebbe7c60ad3e8d4a13f573712,5,0,1,2017-05-19,-8.0,credit_card,fashion_bolsas_e_acessorios
118297,387c2c33e495538e94878a500bf8e6d5,1.0,credit_card,1.0,87.58,27f7d35941527e3ab4db13c6fe2a5e77,d8ce0e44ece69672d6bb5dafabb21aa5,RJ,1.0,6a0713a63e990052795623e76cc2995e,cama_mesa_banho,ee9210128e9071190196956a76fba623,1,0,1,2017-04-22,-20.0,credit_card,cama_mesa_banho


In [ ]:
customer_df1 = customer_df.set_index('customer_unique_id')['customer_state']
customer_df1

In [ ]:
df1 = pd.merge(df1, customer_df[['customer_unique_id', 'customer_state']], how='left', on='customer_unique_id')
df1

In [ ]:
df1 = pd.merge(df1, customer_df.set_index('customer_unique_id')['customer_state'], left_on='customer_unique_id', right_index=True)
df1

In [ ]:
df1 = pd.get_dummies(df1)
df1

In [ ]:
df2 = poc2_df.groupby('customer_unique_id').agg({'payment_value':'sum'})
df2

In [ ]:
common_cust = pd.merge(df1, df2, on='customer_unique_id')
common_cust.info()
common_cust.head()

In [ ]:
plt.scatter(common_cust.review_score, common_cust.payment_value)

In [ ]:
plt.scatter(common_cust.payment_installments, common_cust.payment_value)

In [ ]:
plt.scatter(common_cust.delay, common_cust.payment_value)

In [ ]:
boleto_series = common_cust.loc[common_cust.payment_type_boleto==1, 'payment_value']
credit_series = common_cust.loc[common_cust.payment_type_credit_card==1, 'payment_value']
debit_series = common_cust.loc[common_cust.payment_type_debit_card==1, 'payment_value']
voucher_series = common_cust.loc[common_cust.payment_type_voucher==1, 'payment_value']

type_dict = {}
type_dict['boleto'] = boleto_series.sum()/boleto_series.count()
type_dict['credit'] = credit_series.sum()/credit_series.count()
type_dict['debit'] = debit_series.sum()/debit_series.count()
type_dict['voucher'] = voucher_series.sum()/voucher_series.count()

type_dict

In [ ]:
X = common_cust.iloc[:,:-1]
y = common_cust.iloc[:,-1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# logistic regression 
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
print('Train Accuracy: ', lr.score(X_train, y_train))
print('Test Accuracy: ', lr.score(X_test, y_test))

from sklearn.model_selection import cross_val_score
print('CV Score:', np.mean(cross_val_score(lr, X_train, y_train, cv = 5)))

In [ ]:
rfm1_df = poc1_df.groupby('customer_unique_id').agg({'purchase_date':'max', 'order_id':'count', 'payment_value':'sum'})
rfm1_df.columns = ['recency', 'frequency', 'monetary']
rfm1_df

In [ ]:
rfm1_df.recency = rfm1_df.recency.apply(lambda x: (poc1_df.purchase_date.max()-x).days)
rfm1_df

In [ ]:
#rfm1_df = pd.read_csv('rfm3m.csv', index_col=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(rfm1_df)
X_scaled = scaler.transform(rfm1_df)
rfm1_km = KMeans(n_clusters=4).fit(X_scaled)

In [ ]:
plt.scatter(rfm1_df.recency, rfm1_df.monetary, c=rfm1_km.labels_)

In [ ]:
plt.scatter(rfm1_df.frequency, rfm1_df.monetary, c=rfm1_km.labels_)

In [ ]:
rfm2_df = poc2_df.groupby('customer_unique_id').agg({'purchase_date':'max', 'order_id':'count', 'payment_value':'sum'})
rfm2_df.columns = ['recency', 'frequency', 'monetary']
rfm2_df

In [ ]:
m2 = rfm2_df[['monetary']]

In [ ]:
clust1_df = pd.DataFrame()
clust1_df['customer_unique_id'] = rfm1_df.index
clust1_df['segment'] = rfm1_km.labels_
clust1_df

In [ ]:
common_cust = pd.merge(clust1_df, m2, on='customer_unique_id')
common_cust

In [ ]:
X = np.array(common_cust.segment).reshape(-1, 1)
y = common_cust.monetary

In [ ]:
plt.scatter(X, y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(X_train, y_train)
print(lr.score(X_train, y_train))
print(lr.score(X_test, y_test))

In [ ]:
lr.coef_

In [ ]:
rfm1_m2_df = pd.merge(rfm1_df, m2, left_index=True, right_index=True)
rfm1_m2_df

In [ ]:
plt.scatter(rfm1_m2_df.recency, rfm1_m2_df.monetary_y)

In [ ]:
plt.scatter(rfm1_m2_df.frequency, rfm1_m2_df.monetary_y)

In [ ]:
plt.scatter(rfm1_m2_df.monetary_x, rfm1_m2_df.monetary_y)

In [ ]:
X = rfm1_m2_df.iloc[:,:-1]
y = rfm1_m2_df.iloc[:,-1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# logistic regression 
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
print('Train Accuracy: ', lr.score(X_train, y_train))
print('Test Accuracy: ', lr.score(X_test, y_test))

In [ ]:
lr.coef_

In [ ]:
common_cust[['monetary']]

In [ ]:
m6m_km = KMeans(n_clusters=2)
m6m_km.fit(common_cust[['monetary']])
plt.scatter(common_cust.monetary, np.zeros([1,common_cust.shape[0]]), c=m6m_km.labels_)

In [ ]:
1-m6m_km.labels_.mean()

In [ ]:
plt.scatter(common_cust.rfm_clust_3m, m6m_km.labels_)

In [ ]:
from sklearn.linear_model import LogisticRegression 
clust_lr = LogisticRegression().fit(common_cust[['rfm_clust_3m']], m6m_km.labels_)

In [ ]:
clust_lr.score(common_cust[['rfm_clust_3m']], m6m_km.labels_)